In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import keras.backend as K
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf

from scipy import stats
from plotly.subplots import make_subplots
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.regularizers import l1_l2
from keras.optimizers import Adam

### Cargamos los datos de entrenamiento y de prueba

##### Los archivos csv los podemos encontrar en la pagina de kaggle

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=train.csv
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=test.csv

In [ ]:

df_train = pd.read_csv('/content/data/train.csv')
df_train = df_train.drop(['Id'], axis=1)
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df_test = pd.read_csv('/content/data/test.csv')
ids = df_test['Id']
df_test = df_test.drop(['Id'], axis=1)
df_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### Veamos el numero de elementos vacios que existen por columna

In [ ]:
(df_train.isna().sum() + (df_train == 'None').sum()).sort_values(ascending=False)

PoolQC         1453
MiscFeature    1406
Alley          1369
Fence          1179
MasVnrType      872
               ... 
Heating           0
HeatingQC         0
MSZoning          0
1stFlrSF          0
SalePrice         0
Length: 80, dtype: int64

Como podemos observar hay columnas que poseen demasiados valores vacios, para resolver esto, eliminaremos las columnas que tengan un excedente de valores vacios, y eliminaremos las filas para aquellas que no representen un gran numero de registros, este numero sera elegido arbitrariamente, pero siempre intentando no eliminar demasiadas filas, para este caso usaremos 50

### Eliminaremos las columnas que tengan mas de 50 valores vacios

In [ ]:
col_drops = df_train.columns[(df_train.isna().sum()>50) | ((df_train == 'None').sum()>50)]
df_train = df_train.drop(col_drops, axis=1)
df_test = df_test.drop(col_drops, axis=1)
label = 'SalePrice'

Eliminamos las columnas tanto del conjunto de entrenamiento como el de prueba, pues necesitaremos las mismas columnas para la prediccion
___

Ahora obtendremos las variables numericas como las variables categoricas, para posteriormente aplicarle cierto tratamiento de datos

In [ ]:
# Variables categoricas
categoricals = df_train.select_dtypes(include = ['object'])
categoricals.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal


In [ ]:
# Variables numericas
numerics = df_train.select_dtypes(include = ['float64', 'int64'])
numerics.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,8450,7,5,2003,2003,196.0,706,0,150,...,0,61,0,0,0,0,0,2,2008,208500
1,20,9600,6,8,1976,1976,0.0,978,0,284,...,298,0,0,0,0,0,0,5,2007,181500
2,60,11250,7,5,2001,2002,162.0,486,0,434,...,0,42,0,0,0,0,0,9,2008,223500
3,70,9550,7,5,1915,1970,0.0,216,0,540,...,0,35,272,0,0,0,0,2,2006,140000
4,60,14260,8,5,2000,2000,350.0,655,0,490,...,192,84,0,0,0,0,0,12,2008,250000


### Histogramas de variables categoricas

In [ ]:
fig = make_subplots(rows=11, cols=3, subplot_titles=categoricals.columns)

for i, categ in enumerate(categoricals.columns):
    fig.add_trace(go.Histogram(x=df_train[categ]), row=i//3+1, col=i%3+1)

fig.update_layout(height=6000, width=900, title_text="Histogramas variables categoricas", showlegend=False)
fig.show()

### Histogramas de variables numericas

In [ ]:
fig = make_subplots(rows=12, cols=3, subplot_titles=numerics.columns)

for i, num in enumerate(numerics):
    fig.add_trace(go.Scatter(x=df_train[num], y=df_train['SalePrice'], mode='markers'), row=i//3+1, col=i%3+1)

fig.update_layout(height=6000, width=900, title_text="Histogramas variables numericas", showlegend=False)
fig.show()

In [ ]:
matrix_corr = df_train.corr(numeric_only=True)

fig = go.Figure(data=go.Heatmap(
                   z=matrix_corr,
                   x=matrix_corr.columns,
                   y=matrix_corr.columns,
                   hoverongaps = False))
fig.show()

### Tratamiento de datos

Cuando necesitamos convertir variables numéricas en variables categóricas, existen varios métodos que podemos utilizar, como el "One-Hot Encoder" o el "Label Encoder".

El "One-Hot Encoder" genera una columna binaria por cada clase presente en nuestra variable. Cada columna representa una clase y contiene un valor de 1 si la observación pertenece a esa clase y un valor de 0 en caso contrario. En otras palabras, el "One-Hot Encoder" crea una representación de variables categóricas en la que cada clase se trata como una entidad distinta e independiente.

Por otro lado, el método "Label Encoder" no genera nuevas columnas, pero asigna un valor numérico discreto a cada clase en la variable. Cada clase se mapea a un número entero único, lo que permite representar la variable categórica con valores numéricos.

La elección entre el "One-Hot Encoder" y el "Label Encoder" depende del contexto y del tipo de análisis que se vaya a realizar. Si se desea tratar cada clase como una entidad independiente y se necesita preservar la información sobre la pertenencia a una clase específica, el "One-Hot Encoder" es más adecuado. Por otro lado, si solo se necesita una representación numérica discreta de las clases y no es necesario preservar la información de pertenencia a una clase específica, entonces el "Label Encoder" puede ser suficiente.

En resumen, tanto el "One-Hot Encoder" como el "Label Encoder" son métodos útiles para convertir variables numéricas en variables categóricas. La elección entre ellos depende del contexto y de los requisitos específicos del análisis que se va a realizar.

En este caso en partular utilizaremos "Label Encoder" puesto que es mas sencillo y cumple el proposito del notebook

In [ ]:
# Convirtiendo variables categoricas a numericas usando el metodo label encoder
for categ in categoricals.columns:
  tr = df_train[categ].notnull().unique().tolist()
  te = df_test[categ].notnull().unique().tolist()

  le = LabelEncoder()
  le.fit(list(set(tr + te)))

  df_train[categ][df_train[categ].notnull()] = le.transform(df_train[categ])
  df_test[categ][df_test[categ].notnull()] = le.transform(df_test[categ])


En estadística, la imputación de datos es el proceso de reemplazar los valores faltantes en un conjunto de datos con nuevos valores estimados. Este paso es crucial antes de entrenar un modelo, ya que los modelos de machine learning no pueden funcionar con valores faltantes.

Existen diferentes métodos disponibles para la imputación de datos. Algunos de los métodos más comunes incluyen la imputación por estadístico, donde se utilizan medidas como la media, la mediana o la moda para reemplazar los valores faltantes. Otro enfoque es la imputación por valor al azar, donde se generan valores aleatorios dentro de un rango adecuado para llenar los espacios vacíos. También está la imputación por vecino más cercano, que se basa en encontrar observaciones similares y utilizar sus valores para imputar los faltantes. Incluso se puede utilizar un modelo de regresión para imputar los valores faltantes, donde se ajusta un modelo basado en las variables predictoras para estimar los valores desconocidos.

En este ejemplo particular, se utilizará la imputación por vecino más cercano. Este método es fácil de implementar y preserva la estructura de los datos, ya que se basa en valores reales observados en lugar de supuestos estadísticos. Sin embargo, es importante tener en cuenta sus limitaciones. Puede no funcionar bien cuando no hay suficientes observaciones similares o cuando los datos tienen alta dimensionalidad. Además, si se seleccionan vecinos atípicos o no representativos, puede introducir ruido en los datos imputados.

In [ ]:
# Imputando valores faltantes
imputer = KNNImputer(n_neighbors=1)

df_train = pd.DataFrame(imputer.fit_transform(df_train), columns=df_train.columns)
df_test = pd.DataFrame(imputer.fit_transform(df_test), columns=df_test.columns)

In [ ]:
# Estandarizando variables numericas no ordinales
for num in numerics.columns:
    if num != label and len(df_train[num].unique()) > 20:
        df_train[num] = stats.zscore(df_train[num])
        df_test[num] = stats.zscore(df_test[num])

In [ ]:
df_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60.0,1.0,-0.207142,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.359325,0.0,-0.270208,0.0,-0.087688,2.0,2008.0,1.0,0.0,208500.0
1,20.0,1.0,-0.091886,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.359325,0.0,-0.270208,0.0,-0.087688,5.0,2007.0,1.0,0.0,181500.0
2,60.0,1.0,0.073480,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.359325,0.0,-0.270208,0.0,-0.087688,9.0,2008.0,1.0,0.0,223500.0
3,70.0,1.0,-0.096897,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,4.092524,0.0,-0.270208,0.0,-0.087688,2.0,2006.0,1.0,0.0,140000.0
4,60.0,1.0,0.375148,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.359325,0.0,-0.270208,0.0,-0.087688,12.0,2008.0,1.0,0.0,250000.0


## Creando red neuronal simple

Antes de entrenar un modelo de aprendizaje automático, es esencial dividir el conjunto de datos en dos subconjuntos: uno de entrenamiento y otro de validación. Esta práctica se conoce como validación cruzada o particionamiento de datos. La proporción comúnmente utilizada para generar el conjunto de validación es del 25% al 33% del conjunto de datos total, aunque esto puede variar dependiendo del tamaño y la naturaleza del conjunto de datos.

La división del conjunto de datos en entrenamiento y validación es fundamental para evaluar la capacidad de generalización del modelo. Cuando entrenamos un modelo, el objetivo es que aprenda a partir de los datos disponibles y pueda aplicar ese conocimiento para hacer predicciones precisas en nuevos datos no vistos previamente. El conjunto de entrenamiento se utiliza para ajustar los pesos del modelo y encontrar la mejor configuración que minimice el error en esos datos.

Una vez que el modelo ha sido entrenado, se utiliza el conjunto de validación para evaluar su rendimiento y medir su capacidad de generalización. El conjunto de validación se compone de datos que el modelo no ha visto durante el entrenamiento y, por lo tanto, proporciona una evaluación imparcial de cómo el modelo se comporta en situaciones de la vida real.

Si no se dividiera el conjunto de datos y se utilizara el conjunto completo para el entrenamiento, podría haber un sesgo de sobreajuste. El sobreajuste ocurre cuando el modelo se ajusta demasiado a los datos de entrenamiento y no logra generalizar bien en datos nuevos. Esto resultaría en un modelo que tiene un rendimiento deficiente en datos no vistos previamente.

La división del conjunto de datos en entrenamiento y validación permite realizar un ajuste adecuado del modelo durante el entrenamiento y luego evaluar su rendimiento en datos no utilizados previamente. Al utilizar un conjunto de validación, se pueden realizar ajustes en la arquitectura del modelo, hiperparámetros u otras configuraciones para mejorar su rendimiento antes de someterlo a pruebas en un conjunto completamente nuevo.

In [ ]:
X = df_train.drop([label], axis=1)
y = df_train[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023)

La elección de la arquitectura del modelo es un aspecto clave en el desarrollo de un sistema de aprendizaje automático. En este caso, se ha decidido utilizar 6 capas ocultas con 256 neuronas cada una. Cada capa oculta utiliza una función de activación ReLU, que es una opción comúnmente utilizada en redes neuronales debido a su capacidad para manejar el problema del desvanecimiento del gradiente y acelerar el entrenamiento.

La elección de 6 capas ocultas y 256 neuronas en cada capa es una decisión basada en la complejidad y dimensionalidad del problema que se está abordando. En problemas más complejos o con una gran cantidad de características, puede ser beneficioso aumentar el número de capas y neuronas para permitir que el modelo aprenda representaciones más sofisticadas y complejas.

La capa de salida del modelo utiliza una activación lineal debido a que el problema se trata de una regresión. En los problemas de regresión, el objetivo es predecir un valor continuo en lugar de una clase discreta, por lo que una activación lineal es apropiada.

El tamaño de batch utilizado durante el entrenamiento es de 16. El tamaño de batch se refiere al número de ejemplos de entrenamiento que se utilizan en cada iteración para calcular el gradiente y actualizar los pesos del modelo. Utilizar un tamaño de batch más grande puede acelerar el proceso de entrenamiento al procesar más ejemplos simultáneamente, pero también puede requerir más memoria. El tamaño de batch de 16 parece ser una elección razonable que equilibra la eficiencia y la capacidad de memoria.

El modelo se ha entrenado durante 5000 épocas. Una época se define como una pasada completa a través de todo el conjunto de entrenamiento. Entrenar durante múltiples épocas permite al modelo ajustarse mejor a los datos y mejorar su rendimiento a medida que aprende patrones y relaciones más complejas. Sin embargo, es importante tener en cuenta que entrenar durante demasiadas épocas también puede llevar al sobreajuste, donde el modelo memoriza los datos de entrenamiento y no generaliza bien a nuevos datos.

Para medir el rendimiento del modelo se ha elegido el error cuadrático medio (MSE, por sus siglas en inglés). El MSE es una métrica comúnmente utilizada en problemas de regresión y penaliza con mayor fuerza los valores atípicos debido a que los errores se elevan al cuadrado. Esto significa que los errores más grandes tienen un impacto desproporcionadamente mayor en la puntuación final de MSE, lo que puede ser útil cuando se desea penalizar fuertemente los errores significativos o anomalías en los datos.

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoints/epoch{epoch:03d}.h5', save_weights_only=True, save_freq=1)

epochs = 5000
batch_size = 64


model = Sequential()

model.add(Dense(units=256, activation='relu', input_dim=X.shape[1]))
model.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=1, activation='linear'))


model.compile(loss='MSE', optimizer=Adam(learning_rate=0.00007), metrics=['MSE'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               17408     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dense_6 (Dense)             (None, 1)                 2

In [ ]:
history = model.fit(
    X_train,
    y_train ,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint],
    verbose=0)

Una forma efectiva de seleccionar la mejor época de un modelo es utilizar una gráfica de validación. Esta gráfica muestra el error del modelo en el conjunto de entrenamiento y el error de validación a lo largo de las épocas de entrenamiento.

El error de entrenamiento se refiere al error que el modelo comete al predecir los valores en el conjunto de entrenamiento. A medida que el modelo se entrena, se espera que el error de entrenamiento disminuya gradualmente, ya que el modelo se ajusta a los datos de entrenamiento.

Por otro lado, el error de validación se refiere al error que el modelo comete al predecir los valores en el conjunto de validación, que consta de datos no vistos durante el entrenamiento. El objetivo es que el modelo generalice bien en nuevos datos y, por lo tanto, se espera que el error de validación disminuya a medida que el modelo aprende patrones y relaciones en los datos de entrenamiento.

Al trazar una gráfica del error de entrenamiento y el error de validación en función de las épocas de entrenamiento, podemos visualizar cómo se está desempeñando el modelo a medida que progresa el entrenamiento. Generalmente, al principio del entrenamiento, tanto el error de entrenamiento como el de validación son altos, ya que el modelo aún no ha aprendido los patrones relevantes en los datos. A medida que el entrenamiento continúa, se espera que ambos errores disminuyan.

Sin embargo, es posible que en algún punto el error de entrenamiento siga disminuyendo, pero el error de validación comience a aumentar nuevamente. Esto indica que el modelo está comenzando a sobreajustarse a los datos de entrenamiento y no generaliza bien en nuevos datos. Esta situación se conoce como sobreajuste y puede resultar en un rendimiento deficiente en datos no vistos previamente.

La mejor época para nuestro modelo sera aquella en la que el error de validación es mínimo. Esto indica que el modelo ha aprendido patrones y relaciones relevantes sin sobreajustarse demasiado a los datos de entrenamiento. Al detener el entrenamiento en este punto, se espera obtener un modelo con una buena capacidad de generalización.

Es importante destacar que la elección de la mejor época puede variar según el conjunto de datos y el problema específico que se esté abordando. Por lo tanto, es recomendable examinar detenidamente la gráfica de validación y seleccionar la época que resulte en un error de validación aceptable y un buen equilibrio entre el rendimiento en el conjunto de entrenamiento y la capacidad de generalización.

In [ ]:
# Mostrando la grafica de validacion
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=history.history['loss'][::50],
                    mode='lines+markers',
                    name='Entrenamiento'))
fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=history.history['val_loss'][::50],
                    mode='lines+markers',
                    name='Validacion'))
fig.show()

In [ ]:
opt = history.history['val_loss'].index(min(history.history['val_loss'])) +1
model.load_weights(f'checkpoints/epoch{opt}.h5')

Una vez que se ha entrenado el modelo y se ha seleccionado la mejor época utilizando técnicas como la gráfica de validación, se está listo para generar las predicciones finales. Estas predicciones se utilizarán para ser calificadas en una competencia o desafío de Kaggle.

Antes de generar las predicciones finales, es importante asegurarse de que los datos de prueba estén en el formato adecuado y sigan las mismas transformaciones que se aplicaron al conjunto de entrenamiento durante el preprocesamiento de datos. Esto garantiza que el modelo pueda hacer predicciones consistentes y precisas en los datos de prueba.

In [ ]:
predicts = model.predict(df_test)
pd.DataFrame({'Id': ids, 'SalePrice': predicts.flatten()}).to_csv('Simple_NN.csv', index=False)

46/46 [==============================] - 0s 3ms/step


### Conlusiones

El desarrollo de una red neuronal simple para problemas de regresión puede ser facilitado por el uso de librerías y herramientas disponibles en el ámbito del aprendizaje automático. Estas herramientas, como TensorFlow, PyTorch o Keras, proporcionan una interfaz sencilla y funciones predefinidas que simplifican la implementación de redes neuronales.

Sin embargo, es crucial tener un conocimiento sólido de los fundamentos teóricos subyacentes para poder abordar desafíos y resolver problemas que puedan surgir durante el proceso de desarrollo. Comprender los conceptos básicos de las redes neuronales, las funciones de activación, las arquitecturas, los hiperparámetros y las técnicas de entrenamiento es esencial para tomar decisiones informadas y obtener resultados óptimos.

Además, es importante destacar que el desarrollo de modelos de aprendizaje automático no se limita únicamente a la implementación de redes neuronales. El proceso completo implica etapas como la exploración y preprocesamiento de datos, la selección adecuada de métricas de evaluación, la validación cruzada, la optimización de hiperparámetros y la interpretación de los resultados.